# Homework 4: Practical analysis with BioPython

For the homework, you are going to extend the code from the analysis of our FASTQ file in lectures 8 and 9.
Recall that the FASTQ file contains reads from a real sequencing run of influenza virus HA and NA genes.

The _actual sequences_ are as follows:

    5'-[end of HA]-AGGCGGCCGC-[16 X N barcode]-3'
    
or 

    5'-[end of NA]-AGGCGGCCGC-[16 X N barcode]-3'
    
The end of NA is:

    ...CACGATAGATAAATAATAGTGCACCAT
    
The end of HA is:

    ...CCGGATTTGCATATAATGATGCACCAT
    
The _sequencing reads_ (located in the FASTQ file) are from the reverse end of these actual sequences, so the first thing in the sequencing reads is the reverse complement of the barcode followed by the reverse complement of the constant sequence, etc.
The reads can originate from **either** HA or NA, and that will be distinguished by the most 3' end of the read.
But in our example exercise in class, we did not distinguish among reads matching to HA and NA, as we didn't even look far enough into the read to tell the identity.

For the homework, your goal is to write code that extends the material from lectures 8 and 9 to also distinguish between HA and NA.
This homework can be completed almost entirely by re-using code from lecture 9. You will need to set up your analysis to do the following:
 1. Get the reverse complement of each read.
 2. Determine if it matches the expected pattern for HA and NA, and if so which one.
 3. If it matches, extract the barcode and add it to a dictionary to keep track of counts.
 4. Determine the number and distribution of barcodes for HA and NA separately.

Please include code to address each of the following questions. Please include code comments to explain what your code is attempting to accomplish. Don't forget to include references to the sources you used to obtain your answer, including your classmates (if you are working in groups).  

1. How many reads map to HA, and how many reads map to NA?

In [1]:
#import packages that are needed
import re
import Bio.SeqIO
import os
os.chdir('/workspaces/tfcb_2022/homeworks/homework04') #make sure in the correct working directory to parse the fastq file

seqreads = list(Bio.SeqIO.parse('barcodes_R1.fastq', 'fastq')) #read in sequencing reads

#convert seqreads which are SeqRecord objects into list of strings with just the sequences
seqreads_str = [] #empty string
for seqrecord in seqreads: #looping through entries in seqreads
    seqreads_str.append(str(seqrecord.seq)) #take the sequence only of each seqread sequence and add to the list

#function to find the reverse complement of a sequence
def reverse_complement(seq, unk_partner = 'N'): #accounts for errors or unknown in sequence with N
    base_partner = {'A':'T', 'T':'A', 'C':'G', 'G':'C'} #dictionary with normal base pair partners
    rseq = '' #empty string
    for base in seq: #loop through the sequence base by base
        if base in base_partner: #if the base is in the base partner dictionary
            rseq = base_partner[base] + rseq #look up the complementary base in the dictionary and add in reverse order
        else: #if not ATCG
            rseq = unk_partner + rseq #add N in its place
    return rseq

#function to read the barcodes while diffrentiating HA and NA
def read_barcode(seqread, bclen, proteinseq, upstream='AGGCGGCCGC'): #similar to code we wrote in class but with extra protein argument
    seqread = seqread.upper() #make sequence all uppercase
    reverse = reverse_complement(seqread) # get the reverse complement of the read

    #compile the barcode search pattern accounting for HA or NA difference with proteinseq while searching at the end of the sequence
    barcode_pattern = re.compile(proteinseq + upstream + f'(?P<barcode>[ATCG]{{{bclen}}})$') #barcode search pattern group that includes all bases with barcode length corresponding to bclen argument

    #search reverse sequences for given barcode pattern
    match = barcode_pattern.search(reverse)

    if match: #if there is a barcode
        barcode = match.group('barcode') #assign the barcode string to barcode
        return barcode #function will return the barcode for the sequence
    else: #if no match, nothing
        return None

barcode_counts_ha = {} #empty dictionary for HA barcode counts
barcode_counts_na = {} #empty dictionary for NA barcode counts

for seq in seqreads_str: # iterate through all sequence reads in the seqreads_str list
    barcode_ha = read_barcode(seq, bclen = 16, proteinseq='CCGGATTTGCATATAATGATGCACCAT') #use the read_barcode function to find barcode for specifically HA using proteinseq as the HA sequence
    barcode_na = read_barcode(seq, bclen = 16, proteinseq='CACGATAGATAAATAATAGTGCACCAT') #use the read_barcode function to find barcode for specifically NA using proteinseq as the NA sequence
    if barcode_ha: # if there is a valid HA barcode
        if barcode_ha in barcode_counts_ha: #if the HA barcode is in the HA barcode counts dictionary
            barcode_counts_ha[barcode_ha] += 1 #add 1 to the value for that HA barcode key
        else: #if HA barcode is not in the HA barcode counts dictionary
            barcode_counts_ha[barcode_ha] = 1 #assign the value 1 for the HA barcode key and add to the dictionary
    if barcode_na: #same thing as above but for the NA barcodes
        if barcode_na in barcode_counts_na:
            barcode_counts_na[barcode_na] += 1
        else:
            barcode_counts_na[barcode_na] = 1

ha_counts = sum(barcode_counts_ha.values()) #add the values in the HA barcode counts dictionaries together to get number of reads that mapped to HA
na_counts = sum(barcode_counts_na.values()) #add the values in the NA barcode counts dictionaries together to get number of reads that mapped to NA

print(f'The number of reads that mapped to HA were {ha_counts}.')
print(f'The number of reads that mapped to NA were {na_counts}.')

The number of reads that mapped to HA were 5245.
The number of reads that mapped to NA were 3907.


2. What is the HA barcode with the most counts (and how many counts)? Also answer the same question for NA.

    _Hint: you will need to find the key associated with the maximum value in your dictionary. There are many ways to do this._

In [2]:
#max function code adapted from https://pythonguides.com/python-find-max-value-in-a-dictionary/
max_ha_barcode = max(barcode_counts_ha, key=barcode_counts_ha.get) #assigns the key(barcode) with the highest number of counts in HA barcode counts dictionary
max_ha_count = max(barcode_counts_ha.values()) #assigns the highest count value in the HA barcode counts dictionary
max_na_barcode = max(barcode_counts_na, key=barcode_counts_na.get) #assigns the key(barcode) with the highest number of counts in NA barcode counts dictionary
max_na_count = max(barcode_counts_na.values()) #assigns the highest count value in the NA barcode counts dictionary
print (f"The HA barcode with the most counts was {max_ha_barcode} with the total counts for this barcode being {max_ha_count}.")
print (f"The HA barcode with the most counts was {max_na_barcode} with the total counts for this barcode being {max_na_count}.")

The HA barcode with the most counts was CCCGACCCGACATTAA with the total counts for this barcode being 155.
The HA barcode with the most counts was ACCAGTTCTCCCCGGG with the total counts for this barcode being 152.
